In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

# 글 목록 정보

In [11]:
def parse_current_page():
    # 페이지 소스를 BeautifulSoup으로 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')
       
    table = soup.find('table', class_="tbl_list")
    table_html = str(table)
    table_df_list = pd.read_html(table_html)
    table_df = table_df_list[0]
    
    ## url 가져오기
    no_list = []
    link_list = []
    
    a = soup.find_all('a')
    for i in range(len(a)):
        tmp = str(a[i])
        start_pos = tmp.find('a href="quics')
    
        if start_pos == -1: # 문자 없음
            continue
        else:
            start_pos = start_pos + 8
            end_pos = tmp.find('onclick')-2
            value = tmp[start_pos:end_pos]
            no_list.append(value)
    
    for num in no_list:
        txt = "https://omoney.kbstar.com/"+str(num)
        link_list.append(txt)
    
    url_list = [url.replace('&amp;', '&') for url in link_list] ## url 들어있는 list
    
    ## 마지막에 이상한 url이 들어가서 마지막 url만 제거
    if len(url_list) > len(table_df):
        url_list.pop()
    
    table_df['url'] = url_list
    
    title_list = table_df['제목']
    date_list = table_df['등록일']
    hit_list = table_df['조회수']
    
    # series -> list 변환
    title_list = title_list.to_list()
    date_list = date_list.to_list()
    hit_list = hit_list.to_list()
    
    return title_list, date_list, hit_list, url_list

In [12]:
###==================== 변경 필수 =======================###
base_url = "https://omoney.kbstar.com/quics?page=C042011"
###======================================================###

driver = webdriver.Chrome(executable_path='C:/chromedriver.exe')
driver.get(base_url)

time.sleep(5)

title = []
date = []
hit = []
url = []

print('========================')
print('현재 페이지수 : 1')
print('파싱중')
title,date,hit,url = parse_current_page()
print('파싱 완료')

###==================== 변경 필수 =======================###
n_pages = 21 # 전체 페이지수
###======================================================###

for i in range(2,n_pages+1):
    print('========================')
    print('현재 페이지수 : '+str(i))
    if (i % 10 == 1) and (i > 1): ## 11, 21... 
        turn_page = driver.find_element_by_class_name('pagingBtn.next')
        turn_page.click()
    else:
        page_link = '//*[@id="pageinput'+str(i)+'"]'
        next_page = driver.find_element_by_xpath(page_link)
        next_page.click()
            
    time.sleep(1)
    
    print('파싱중')
    tmp1, tmp2, tmp3, tmp4 = parse_current_page()
    
    title.extend(tmp1)
    date.extend(tmp2)
    hit.extend(tmp3)
    url.extend(tmp4)
    print('파싱 완료')

현재 페이지수 : 1
파싱중
파싱 완료
현재 페이지수 : 2
파싱중
파싱 완료
현재 페이지수 : 3
파싱중
파싱 완료
현재 페이지수 : 4
파싱중
파싱 완료
현재 페이지수 : 5
파싱중
파싱 완료
현재 페이지수 : 6
파싱중
파싱 완료
현재 페이지수 : 7
파싱중
파싱 완료
현재 페이지수 : 8
파싱중
파싱 완료
현재 페이지수 : 9
파싱중
파싱 완료
현재 페이지수 : 10
파싱중
파싱 완료
현재 페이지수 : 11
파싱중
파싱 완료
현재 페이지수 : 12
파싱중
파싱 완료
현재 페이지수 : 13
파싱중
파싱 완료
현재 페이지수 : 14
파싱중
파싱 완료
현재 페이지수 : 15
파싱중
파싱 완료
현재 페이지수 : 16
파싱중
파싱 완료
현재 페이지수 : 17
파싱중
파싱 완료
현재 페이지수 : 18
파싱중
파싱 완료
현재 페이지수 : 19
파싱중
파싱 완료
현재 페이지수 : 20
파싱중
파싱 완료
현재 페이지수 : 21
파싱중
파싱 완료


In [15]:
issue_df = pd.DataFrame()
issue_df['title'] = title
issue_df['date'] = date
issue_df['hit'] = hit
issue_df['url'] = url

In [16]:
issue_df

,title,date,hit,url
0,3 월 FOMC 25bp 금리 인상,2023.03.23,58908,https://omoney.kbstar.com/quics?page=C042011&b...
1,SVB/CS 사태 현황 및 주목할 사항,2023.03.22,50994,https://omoney.kbstar.com/quics?page=C042011&b...
2,연초 원자재 시장 점검,2023.03.08,40458,https://omoney.kbstar.com/quics?page=C042011&b...
3,한은 10개월만에 첫 금리 동결,2023.02.23,70286,https://omoney.kbstar.com/quics?page=C042011&b...
4,달러화 전망 및 투자 시사점,2023.02.17,44386,https://omoney.kbstar.com/quics?page=C042011&b...
...,...,...,...,...
203,주요국 기준금리 인상 시점 점검,2010.03.26,138,https://omoney.kbstar.com/quics?page=C042011&b...
204,중국 지준율 인상과 주식시장 전망,2010.01.18,288,https://omoney.kbstar.com/quics?page=C042011&b...
205,대형주 vs 중소형주 펀드 비교,2010.01.14,330,https://omoney.kbstar.com/quics?page=C042011&b...
206,'2007년 증시 고점과 현재 상황 비교,2009.11.11,435,https://omoney.kbstar.com/quics?page=C042011&b...


In [17]:
issue_df.to_csv("invest_issue.csv", index = False)

In [18]:
df = pd.read_csv('invest_issue.csv')
df.head()

,title,date,hit,url
0,3 월 FOMC 25bp 금리 인상,2023.03.23,58908,https://omoney.kbstar.com/quics?page=C042011&b...
1,SVB/CS 사태 현황 및 주목할 사항,2023.03.22,50994,https://omoney.kbstar.com/quics?page=C042011&b...
2,연초 원자재 시장 점검,2023.03.08,40458,https://omoney.kbstar.com/quics?page=C042011&b...
3,한은 10개월만에 첫 금리 동결,2023.02.23,70286,https://omoney.kbstar.com/quics?page=C042011&b...
4,달러화 전망 및 투자 시사점,2023.02.17,44386,https://omoney.kbstar.com/quics?page=C042011&b...


In [19]:
df['url'][0]

'https://omoney.kbstar.com/quics?page=C042011&boardId=613&compId=b029068&articleId=121286&bbsMode=view&viewPage=1&articleClass=&searchCondition=title&searchStr='

# pdf 다운

In [20]:
content_list = []
pdftitles = []
titles_check = []
link = df['url'][0] # 첫번째 글, 수정
pdf_check=[]

driver = webdriver.Chrome(executable_path='C:/chromedriver.exe')
driver.get(link)

time.sleep(5)

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

# 글 정보 추출
content = soup.find('dd', class_="cont")
content = content.get_text()
content_list.append(content) # 리스트에 추가

# pdf 파일명(추후에 비교 및 확인 목적)
file_name = soup.find('input', {'id': '_FILE_NAME'}).get('value')
pdftitles.append(file_name) # 리스트에 추가

# pdf 다운
driver.find_element_by_css_selector('a[onclick^="OpenFileDownload"]').click()
pdf_check.append('완료')

# 첫번째 글 정보 추출하는 거랑 아래랑 합쳐도 상관없음. url 입력하는 방식으로 바꾼 이후로 코드 수정을 안했음
## ======================수정, 글개수 ====================================
n = 208
## =======================================================================

for i in range(1,n):
    print('========================')
    print(str(i)+'번째')
    
    url = df['url'][i]
    driver.get(url)
    
    time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 글 정보 추출
    content = soup.find('dd', class_="cont")
    content = content.get_text()
    content_list.append(content) # 리스트에 추가
    print('글 추출 완료')

    # pdf 다운
    try:
        file_name = soup.find('input', {'id': '_FILE_NAME'}).get('value')
    except:
        pdftitles.append('없음')
        pdf_check.append('실패')
        print('pdf 다운 실패/없음')
    else:
        pdftitles.append(file_name) # 리스트에 추가
        print('pdf 파일명 추출 완료')
    
        driver.find_element_by_css_selector('a[onclick^="OpenFileDownload"]').send_keys(Keys.ENTER)
        pdf_check.append('완료')
        print('pdf 다운 완료')

1번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
2번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
3번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
4번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
5번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
6번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
7번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
8번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
9번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
10번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
11번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
12번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
13번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
14번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
15번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
16번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
17번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
18번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
19번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
20번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
21번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
22번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
23번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
24번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
25번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
26번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
27번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
28번째
글 추출 

글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
134번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
135번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
136번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
137번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
138번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
139번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
140번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
141번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
142번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
143번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
144번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
145번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
146번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
147번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
148번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
149번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
150번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
151번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
152번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
153번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
154번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
155번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
156번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
157번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
158번째
글 추출 완료
pdf 파일명 추출 완료
pdf 다운 완료
159번째
글 추출 완료
pdf 

In [21]:
df_new = pd.DataFrame()

In [22]:
df_new['content'] = content_list
df_new['pdf title'] = pdftitles
df_new['pdf check'] = pdf_check

In [23]:
df_new

,content,pdf title,pdf check
0,\n\n[주요 내용]\n ☞1) 3월 FOMC 금리 인상폭 25bp로 축소\n ...,(이슈리포트)_23년_3월_FOMC.pdf,완료
1,"\n\n[주요 내용]\n ☞1) 미국, 유럽 은행 발 금융 불안 대두\n ☞2)...",실리콘밸리은행_파산_사태_현황_및_주목할_사항.pdf,완료
2,"\n\n[주요 내용]\n ☞1) 1분기 원자재 성과 부진\n ☞2) 원자재, 우...",23년_연초_원자재_시장_점검_1539565331900226.pdf,완료
3,\n\n[주요 내용]\n ☞1) 한국은행 지난 4월 이후 처음으로 기준금리 동결\...,한국은행_2월_기준금리_동결.pdf,완료
4,"\n\n[주요 내용]\n ☞1) 달러 환율 추이\n ☞2) 환율 움직임, 금리 ...",(이슈리포트)_달러화_전망_및_투자_시사점.pdf,완료
...,...,...,...
203,\n\n첨부 파일을 확인하세요.\n\n,주요국 기준금리 인상 시점 전망.pdf,완료
204,\n\n첨부 파일을 확인하세요.\n\n,중국 지준율 인상과 향후 증시 전망.pdf,완료
205,\n\n중소형주와 대형주 펀드를 요약 비교한 보고서입니다.\n\n첨부 파일을 확인...,중소형주.pdf,완료
206,\n\n첨부 파일을 확인하세요.\n\n,|07년 증시 고점과 현재 상황 비교2183439597874213.pdf,완료


In [31]:
pdftitles

['(이슈리포트)_23년_3월_FOMC.pdf',
 '실리콘밸리은행_파산_사태_현황_및_주목할_사항.pdf',
 '23년_연초_원자재_시장_점검_1539565331900226.pdf',
 '한국은행_2월_기준금리_동결.pdf',
 '(이슈리포트)_달러화_전망_및_투자_시사점.pdf',
 'KB_Global_Risk_Watch(23.1Q).pdf',
 '(이슈리포트)_23년_2월_FOMC.pdf',
 '(이슈리포트)_2023년_채권투자_.pdf',
 '글로벌_금융시장_변동성_확대_관련_QnA.pdf',
 'KB_Issue_20220224_5850443671035985.pdf',
 '[시장이슈]_통화긴축_우려로_코스피_2700포인트_하회.pdf',
 '(이슈리포트)중국_규제_리스크_점검.pdf',
 '(이슈리포트)_미국_대선_결과와_시장_영향_점검_1707307080331429.pdf',
 '코로나19_재확산에_따른_금융시장_점검.pdf',
 '코로나19 관련 시장 점검2185372998401313.pdf',
 '(이슈리포트) 코로나 바이러스 확산 영향.pdf',
 '(이슈)중국 우한폐렴 관련 시장점검.pdf',
 '(고객용)한눈에 보는 이슈.pdf',
 '[종합] 포트폴리오 시작하기.pdf',
 '연말 주요시장이슈.pdf',
 '[고객용] 케이봇쌤 전문가 포트폴리오 투자전략.pdf',
 '9월 FOMC 결과.pdf',
 '[이슈보고서] 국내외 증시 주요문의사항_2019.8.9.pdf',
 '[이슈보고서] 국내증시 점검 및 향후 전망.pdf',
 '(이슈리포트_스타알림)무역협상 불확실성 재부각.pdf',
 '(이슈리포트)증시 하락 요인 점검.pdf',
 '국내증시하락요인점검(고객용).pdf',
 '한눈에 보는 터키 이슈.pdf',
 '(코스닥벤처펀드)알아보기.pdf',
 '[이슈리포트]미중 무역분쟁 점검 .pdf',
 '(이슈리포트)3월FOMC리뷰.pdf',
 '국내증시점검(고객용).pdf',
 '[이슈리포트]연초 글로벌 증시 현황 점검124103019778331

In [24]:
df['content'] = content_list
df['pdf title'] = pdftitles
df['pdf check'] = pdf_check

In [25]:
df

,title,date,hit,url,content,pdf title,pdf check
0,3 월 FOMC 25bp 금리 인상,2023.03.23,58908,https://omoney.kbstar.com/quics?page=C042011&b...,\n\n[주요 내용]\n ☞1) 3월 FOMC 금리 인상폭 25bp로 축소\n ...,(이슈리포트)_23년_3월_FOMC.pdf,완료
1,SVB/CS 사태 현황 및 주목할 사항,2023.03.22,50994,https://omoney.kbstar.com/quics?page=C042011&b...,"\n\n[주요 내용]\n ☞1) 미국, 유럽 은행 발 금융 불안 대두\n ☞2)...",실리콘밸리은행_파산_사태_현황_및_주목할_사항.pdf,완료
2,연초 원자재 시장 점검,2023.03.08,40458,https://omoney.kbstar.com/quics?page=C042011&b...,"\n\n[주요 내용]\n ☞1) 1분기 원자재 성과 부진\n ☞2) 원자재, 우...",23년_연초_원자재_시장_점검_1539565331900226.pdf,완료
3,한은 10개월만에 첫 금리 동결,2023.02.23,70286,https://omoney.kbstar.com/quics?page=C042011&b...,\n\n[주요 내용]\n ☞1) 한국은행 지난 4월 이후 처음으로 기준금리 동결\...,한국은행_2월_기준금리_동결.pdf,완료
4,달러화 전망 및 투자 시사점,2023.02.17,44386,https://omoney.kbstar.com/quics?page=C042011&b...,"\n\n[주요 내용]\n ☞1) 달러 환율 추이\n ☞2) 환율 움직임, 금리 ...",(이슈리포트)_달러화_전망_및_투자_시사점.pdf,완료
...,...,...,...,...,...,...,...
203,주요국 기준금리 인상 시점 점검,2010.03.26,138,https://omoney.kbstar.com/quics?page=C042011&b...,\n\n첨부 파일을 확인하세요.\n\n,주요국 기준금리 인상 시점 전망.pdf,완료
204,중국 지준율 인상과 주식시장 전망,2010.01.18,288,https://omoney.kbstar.com/quics?page=C042011&b...,\n\n첨부 파일을 확인하세요.\n\n,중국 지준율 인상과 향후 증시 전망.pdf,완료
205,대형주 vs 중소형주 펀드 비교,2010.01.14,330,https://omoney.kbstar.com/quics?page=C042011&b...,\n\n중소형주와 대형주 펀드를 요약 비교한 보고서입니다.\n\n첨부 파일을 확인...,중소형주.pdf,완료
206,'2007년 증시 고점과 현재 상황 비교,2009.11.11,435,https://omoney.kbstar.com/quics?page=C042011&b...,\n\n첨부 파일을 확인하세요.\n\n,|07년 증시 고점과 현재 상황 비교2183439597874213.pdf,완료


In [26]:
df.to_csv("invest_issue_new.csv", index = False)

## pdf 다시 다운로드

In [32]:
content_list = []
pdftitles = []
titles_check = []
link = df['url'][0] # 첫번째 글, 수정
pdf_check=[]

driver = webdriver.Chrome(executable_path='C:/chromedriver.exe')
driver.get(link)

time.sleep(5)

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

# pdf 다운
driver.find_element_by_css_selector('a[onclick^="OpenFileDownload"]').click()
pdf_check.append('완료')

# 첫번째 글 정보 추출하는 거랑 아래랑 합쳐도 상관없음. url 입력하는 방식으로 바꾼 이후로 코드 수정을 안했음
## ======================수정, 글개수 ====================================
n = 208
## =======================================================================

for i in range(1,n):
    print('========================')
    print(str(i)+'번째')
    
    url = df['url'][i]
    driver.get(url)
    
    time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    driver.find_element_by_css_selector('a[onclick^="OpenFileDownload"]').click()
    pdf_check.append('완료')
    print('pdf 다운 완료')

1번째
pdf 다운 완료
2번째
pdf 다운 완료
3번째
pdf 다운 완료
4번째
pdf 다운 완료
5번째
pdf 다운 완료
6번째
pdf 다운 완료
7번째
pdf 다운 완료
8번째
pdf 다운 완료
9번째
pdf 다운 완료
10번째
pdf 다운 완료
11번째
pdf 다운 완료
12번째
pdf 다운 완료
13번째
pdf 다운 완료
14번째
pdf 다운 완료
15번째
pdf 다운 완료
16번째
pdf 다운 완료
17번째
pdf 다운 완료
18번째
pdf 다운 완료
19번째
pdf 다운 완료
20번째
pdf 다운 완료
21번째
pdf 다운 완료
22번째
pdf 다운 완료
23번째
pdf 다운 완료
24번째
pdf 다운 완료
25번째
pdf 다운 완료
26번째
pdf 다운 완료
27번째
pdf 다운 완료
28번째
pdf 다운 완료
29번째
pdf 다운 완료
30번째
pdf 다운 완료
31번째
pdf 다운 완료
32번째
pdf 다운 완료
33번째
pdf 다운 완료
34번째
pdf 다운 완료
35번째
pdf 다운 완료
36번째
pdf 다운 완료
37번째
pdf 다운 완료
38번째
pdf 다운 완료
39번째
pdf 다운 완료
40번째
pdf 다운 완료
41번째
pdf 다운 완료
42번째
pdf 다운 완료
43번째
pdf 다운 완료
44번째
pdf 다운 완료
45번째
pdf 다운 완료
46번째
pdf 다운 완료
47번째
pdf 다운 완료
48번째
pdf 다운 완료
49번째
pdf 다운 완료
50번째
pdf 다운 완료
51번째
pdf 다운 완료
52번째
pdf 다운 완료
53번째
pdf 다운 완료
54번째
pdf 다운 완료
55번째
pdf 다운 완료
56번째
pdf 다운 완료
57번째
pdf 다운 완료
58번째
pdf 다운 완료
59번째
pdf 다운 완료
60번째
pdf 다운 완료
61번째
pdf 다운 완료
62번째
pdf 다운 완료
63번째
pdf 다운 완료
64번째
pdf 다운 완료
65번째
pdf 다운 완료
66번째
pdf 다운 완료
67번째
pdf 다운 완료
68번째

ElementClickInterceptedException: Message: element click intercepted: Element <a href="#none" onclick="OpenFileDownload('frmDownload1', '이란발_지정학적_리스크와_국제유가_PB_.pdf', 'bbs' ,'22/613', 'Y'); return false;">...</a> is not clickable at point (647, 584). Other element would receive the click: <div class="ui-widget-overlay" style="width: 1470px; height: 1035px; z-index: 10006;"></div>
  (Session info: chrome=115.0.5790.110)


In [37]:
for i in range(30,60):
    print('========================')
    print(str(i)+'번째')
    
    url = df['url'][i]
    driver.get(url)
    
    time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    driver.find_element_by_css_selector('a[onclick^="OpenFileDownload"]').click()
    pdf_check.append('완료')
    print('pdf 다운 완료')

30번째
pdf 다운 완료
31번째
pdf 다운 완료
32번째
pdf 다운 완료
33번째
pdf 다운 완료
34번째
pdf 다운 완료
35번째
pdf 다운 완료
36번째
pdf 다운 완료
37번째
pdf 다운 완료
38번째
pdf 다운 완료
39번째
pdf 다운 완료
40번째
pdf 다운 완료
41번째
pdf 다운 완료
42번째
pdf 다운 완료
43번째
pdf 다운 완료
44번째
pdf 다운 완료
45번째
pdf 다운 완료
46번째
pdf 다운 완료
47번째
pdf 다운 완료
48번째
pdf 다운 완료
49번째
pdf 다운 완료
50번째
pdf 다운 완료
51번째
pdf 다운 완료
52번째
pdf 다운 완료
53번째
pdf 다운 완료
54번째
pdf 다운 완료
55번째
pdf 다운 완료
56번째
pdf 다운 완료
57번째
pdf 다운 완료
58번째
pdf 다운 완료
59번째
pdf 다운 완료


In [38]:
df['url'][30]

'https://omoney.kbstar.com/quics?page=C042011&boardId=613&compId=b029068&articleId=14658&bbsMode=view&viewPage=4&articleClass=&searchCondition=title&searchStr='

In [39]:
59-30

29

In [34]:
for i in range(201,n):
    print('========================')
    print(str(i)+'번째')
    
    url = df['url'][i]
    driver.get(url)
    
    time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    driver.find_element_by_css_selector('a[onclick^="OpenFileDownload"]').click()
    pdf_check.append('완료')
    print('pdf 다운 완료')

201번째
pdf 다운 완료
202번째
pdf 다운 완료
203번째
pdf 다운 완료
204번째
pdf 다운 완료
205번째
pdf 다운 완료
206번째
pdf 다운 완료
207번째
pdf 다운 완료


In [80]:
df['title'][208-175] # pdf 파일 2개 존재

'글로벌 시장 이슈 점검(셀트리온과 국제유가)'

In [71]:
#df[df['title']=='브릭스-인도,러시아 주식시장 전망']
df.loc[110, 'pdf check'] = '실패'
df.loc[110]

title                                       브릭스-인도,러시아 주식시장 전망
date                                                2014.05.12
hit                                                         19
url          https://omoney.kbstar.com/quics?page=C042011&b...
content                            \n\n첨부파일을 참조해 주시기 바랍니다.\n\n
pdf title                Issue Report_.브릭스_인도, 러시아 주식시장 전망.pdf
pdf check                                                   실패
Name: 110, dtype: object

In [72]:
#df[df['title']=='미국 재정절벽 합의, 2013년도 미국 경제는?']
df.loc[112, 'pdf check']='실패'

In [73]:
#df[df['title']=='선진국 경제지표, 정책 update']
df.loc[166,'pdf check']='실패'

In [82]:
df['url'][33]

'https://omoney.kbstar.com/quics?page=C042011&boardId=613&compId=b029068&articleId=14094&bbsMode=view&viewPage=4&articleClass=&searchCondition=title&searchStr='

In [81]:
df[df['title']=='글로벌 시장 이슈 점검(셀트리온과 국제유가)']

,title,date,hit,url,content,pdf title,pdf check
33,글로벌 시장 이슈 점검(셀트리온과 국제유가),2018.01.18,155642,https://omoney.kbstar.com/quics?page=C042011&b...,\n\n1. 셀트리온 이전 상장과 코스닥 시장 전망\n2. 유가전망과 원자재 투자전...,셀트리온 이전상장과 코스닥 시장(고객용).pdf,완료


In [83]:
df.loc[33,'pdf title']='셀트리온 이전상장과 코스닥 시장(고객용).pdf, 유가전망과 원자재 투자전략(고객용).pdf'
df.loc[33, 'pdf check']='2개'

In [84]:
df.to_csv("invest_issue_new.csv", index = False)